In [ ]:
from pathlib import Path
from pkg_resources import resource_filename
from joblib import Parallel, delayed
from joblib.externals.loky import set_loky_pickler
from pathos.pools import ParallelPool
import bids
import numpy as np
import pandas as pd
from contarg.normgrid import (
    get_prob_vine,
    setup_uncert_sims,
    run_clusters,
    calc_stimgrid,
    make_uncert_sims,
    make_uncert_surfaces   
)

In [46]:
#calc_stimgrid??

In [11]:
plot=True
magne_min_percentile = 99.9
uncertainty_fwhm = 2
nbootstraps = 100
block_length = 45
pairwise_sig_thresh = 0.1
sw_thresh = 2
mt_thresh = 60
make_plots=True
stimroi = "expandedcoleBA46"
refroi= "bilateralfullSGCsphere"
min_angle_motor_thresh = 30
maxMT=80
dist_std=2
scalp_res = 2
angle_std=2.5
distancetoscalp=2
overwrite = False


subjects = []
sessions = [1,2]
smoothings = [2.55]
fd_thresh = 0.3

outdir = Path('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2')
outdir.mkdir(exist_ok=True)
anat_dir = outdir / 'anat_preproc'

jobids = {}

fmriprep_root = Path('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/fmriprep-23-2-2/')
bidslayout = bids.BIDSLayout(fmriprep_root,validate=False)
if len(subjects) < 1:
        subjects = bidslayout.get_subjects()
print(subjects)
#print(bidslayout)

['NDARINV1285PMCK', 'NDARINV1EECRFPM', 'NDARINV1EZ26N40', 'NDARINV1H7JEJW1', 'NDARINV2484CB0H', 'NDARINV2BW6G83G', 'NDARINV52XG9LJ3', 'NDARINV6UU2L6YJ', 'NDARINV7A5RDHVW', 'NDARINV7BTZH7L2', 'NDARINV7TV9UT40', 'NDARINV85UUUHN0', 'NDARINVC25XBH2T', 'NDARINVCVZ70012', 'NDARINVD5FWJDCY', 'NDARINVEN1P5RFL', 'NDARINVFPFM6B57', 'NDARINVG8VWC1TN', 'NDARINVGH5DHURH', 'NDARINVLMU4CC2J', 'NDARINVN394NEWK', 'NDARINVP9Y862GP', 'NDARINVPE5JBKJ3', 'NDARINVRZL7PGK1', 'NDARINVU3CF21GV', 'NDARINVU7WWPJ0M', 'NDARINVV104HFJY', 'NDARINVWFN7K4C5', 'NDARINVWP2U8M5Y', 'NDARINVX7KKFJ5E', 'NDARINVXTMW4PWB', 'NDARINVXZRDA6CJ', 'NDARINVYKVPYY2M']


# Run Headmodel

In [48]:
# build headmodels

In [41]:
cmds = []
for hmsub in subjects:
    for session in bidslayout.get_sessions(subject=hmsub):
        #print(session)
        #find T1w and T2w preproc
        filesT1w =  bidslayout.get(subject=hmsub,
                                         session=session,
                                         extension=".nii.gz",
                                         suffix="T1w",
                                         desc="preproc",
                                         space=None)
        #print(filesT1w)
        filesT2w =  bidslayout.get(subject=hmsub,
                                         session=session,
                                         extension=".nii.gz",
                                         suffix="T2w",
                                         desc="preproc",
                                         space=None)
        #print(filesT2w)
        if len(filesT2w) < 1:
            continue # This will skip only T1w since the surfaces are bad.
            ses_out_dir = anat_dir /f'sub-{hmsub}'
            ses_out_dir.mkdir(exist_ok=True, parents=True)
            #proced_anat_dir = liston_root / f'sub-{hmsub}/anat'
            # Will assume the first is the correct.
            t1w_path = Path(filesT1w[0].path)
            #t2w_path = proced_anat_dir / 'T1w/T2w_acpc_dc_restore.nii.gz'
            hm_outdir = ses_out_dir / f"HeadModel/m2m_{hmsub}"
        #if not hm_outdir.exists():
            print(hm_outdir)
            cmd = [
                "contarg",
                "tans",
                "headmodel",
                "--t1",
                t1w_path.as_posix(),
                "--out-dir",
                ses_out_dir.as_posix(),
                "--subject",
                hmsub
            ]
            cmds.append(' '.join(cmd))
            #print(cmds)
        else :
            print(f'using T1w and T2w for {hmsub}')
            ses_out_dir = anat_dir /f'sub-{hmsub}'
            ses_out_dir.mkdir(exist_ok=True, parents=True)
            #proced_anat_dir = liston_root / f'sub-{hmsub}/anat'
            t1w_path = Path(filesT1w[0].path)
            t2w_path = Path(filesT2w[0].path)
            hm_outdir = ses_out_dir / f"HeadModel/m2m_{hmsub}"
            #if not hm_outdir.exists():
            #print(hm_outdir)
            cmd = [
                "contarg",
                "tans",
                "headmodel",
                "--t1",
                t1w_path.as_posix(),
                "--t2",
                t2w_path.as_posix(),
                "--out-dir",
                ses_out_dir.as_posix(),
                "--subject",
                hmsub
            ]
            cmds.append(' '.join(cmd))

using T1w and T2w for NDARINV1285PMCK
using T1w and T2w for NDARINV1EECRFPM
using T1w and T2w for NDARINV1H7JEJW1
using T1w and T2w for NDARINV52XG9LJ3
using T1w and T2w for NDARINV6UU2L6YJ
using T1w and T2w for NDARINV7BTZH7L2
using T1w and T2w for NDARINV7TV9UT40
using T1w and T2w for NDARINV85UUUHN0
using T1w and T2w for NDARINVC25XBH2T
using T1w and T2w for NDARINVCVZ70012
using T1w and T2w for NDARINVD5FWJDCY
using T1w and T2w for NDARINVFPFM6B57
using T1w and T2w for NDARINVG8VWC1TN
using T1w and T2w for NDARINVGH5DHURH
using T1w and T2w for NDARINVLMU4CC2J
using T1w and T2w for NDARINVN394NEWK
using T1w and T2w for NDARINVP9Y862GP
using T1w and T2w for NDARINVPE5JBKJ3
using T1w and T2w for NDARINVRZL7PGK1
using T1w and T2w for NDARINVV104HFJY
using T1w and T2w for NDARINVWFN7K4C5
using T1w and T2w for NDARINVWP2U8M5Y
using T1w and T2w for NDARINVX7KKFJ5E
using T1w and T2w for NDARINVXZRDA6CJ
using T1w and T2w for NDARINVYKVPYY2M


In [42]:
print(len(cmds))
print(cmds[24])

25
contarg tans headmodel --t1 /data/ENIGMA-MDD/BIDS/ABCD/derivatives/fmriprep-23-2-2/sub-NDARINVYKVPYY2M/ses-4yearfollowupyarm1/anat/sub-NDARINVYKVPYY2M_ses-4yearfollowupyarm1_desc-preproc_T1w.nii.gz --t2 /data/ENIGMA-MDD/BIDS/ABCD/derivatives/fmriprep-23-2-2/sub-NDARINVYKVPYY2M/ses-4yearfollowupyarm1/anat/sub-NDARINVYKVPYY2M_ses-4yearfollowupyarm1_run-24_desc-preproc_T2w.nii.gz --out-dir /data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINVYKVPYY2M --subject NDARINVYKVPYY2M


In [43]:
swarm_cmd_dir = anat_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = anat_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [44]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'headmodels'
    
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'headmodels'
    jobids[run_name] = ! swarm -f {swarm_cmd_file} -g 80 -t 22 --module matlab,freesurfer/6,fsl,simnibs/4.0,connectome-workbench,ANTs  --time 12:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobids[run_name])

['24005064']


# Run simulations for each point on the gyral lip

In [49]:
import nibabel as nib
a = nib.load('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/fmriprep-23-2-2/sub-NDARINV1285PMCK/ses-4yearfollowupyarm1/anat/sub-NDARINV1285PMCK_ses-4yearfollowupyarm1_run-4_hemi-L_space-fsLR_desc-msmsulc_sphere.surf.gii')
b = nib.load('/home/zugmana2/.cache/templateflow/tpl-fsLR/tpl-fsLR_hemi-L_den-32k_sphere.surf.gii')
c = nib.load('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/fmriprep-23-2-2/sub-NDARINV1285PMCK/ses-4yearfollowupyarm1/anat/sub-NDARINV1285PMCK_ses-4yearfollowupyarm1_run-4_hemi-L_midthickness.surf.gii')

In [50]:
print(a.agg_data()[0].shape)
print(b.agg_data()[0].shape)
print(c.agg_data()[0].shape)

(135760, 3)
(32492, 3)
(135760, 3)


In [52]:
#anat_dir

for hmsub in subjects:
    #if hmsub in ["NDARINVU7WWPJ0M"]:#"NDARINV1H7JEJW1"]:#,"NDARINV2484CB0H","NDARINV2BW6G83G","NDARINV7BTZH7L2","NDARINV85UUUHN0","NDARINVV104HFJY"]:
    #    continue
    for session in bidslayout.get_sessions(subject=hmsub):
        #print(session)
        if not Path(anat_dir / f'sub-{hmsub}').exists():
            continue
        filesT1w =  bidslayout.get(subject=hmsub,
                                         session=session,
                                         extension=".nii.gz",
                                         suffix="T1w",
                                         desc="preproc",
                                         space=None)
        fmriprepanat = filesT1w[0].dirname
        #print(fmriprepanat)
        src_surf_dir = ''
        surf_info_dir = ''
        grid_out_dir = anat_dir / f'sub-{hmsub}/SearchGrid'
        grid_out_dir.mkdir(exist_ok=True, parents=True)
        grid_out = grid_out_dir / 'SearchGrid.npy'
        grid_out_figs = grid_out_dir / 'figures'
        grid_out_figs.mkdir(exist_ok=True, parents=True)
    
        headmodel_dir = anat_dir /f'sub-{hmsub}/HeadModel'
        sim_out_dir = anat_dir / f'sub-{hmsub}/Simulation'
        # The anat dir here is the fmriprep anat?
 
        print(headmodel_dir)
        try:
            #stimgrid = calc_stimgrid(subject, src_surf_dir, surf_info_dir, headmodel_dir, grid_out_dir, make_plots=True, overwrite=True)
            stimgrid = calc_stimgrid(hmsub, src_surf_dir, surf_info_dir,
                       headmodel_dir,
                       grid_out_dir, make_plots=True, stimroi="expandedcoleBA46",
                       refroi= "bilateralfullSGCsphere", overwrite=True, fmriprep=True,
                      layout=bidslayout, anat_dir=fmriprepanat)
                      
        except ValueError as e:
            print(f"sub-{hmsub} failed with value error:")
            print(e)
        

/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1285PMCK/HeadModel
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1EECRFPM/HeadModel
sub-NDARINV1EECRFPM failed with value error:
Scalp triangles are not arranged systematically
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1H7JEJW1/HeadModel
sub-NDARINV1H7JEJW1 failed with value error:
Scalp triangles are not arranged systematically
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV52XG9LJ3/HeadModel
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV6UU2L6YJ/HeadModel
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV7BTZH7L2/HeadModel
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV7TV9UT40/HeadModel
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV8

In [53]:
cmds = []

for hmsub in subjects:
    #if hmsub in ["NDARINV1H7JEJW1","NDARINV2484CB0H","NDARINV2BW6G83G","NDARINV7BTZH7L2","NDARINV85UUUHN0","NDARINVV104HFJY"]:
    #    continue
    for session in bidslayout.get_sessions(subject=hmsub):
        #print(session)
        if not Path(anat_dir / f'sub-{hmsub}').exists():
            continue
        print(hmsub)
        filesT1w =  bidslayout.get(subject=hmsub,
                                         session=session,
                                         extension=".nii.gz",
                                         suffix="T1w",
                                         desc="preproc",
                                         space=None)
        fmriprepanat = filesT1w[0].dirname
        #print(fmriprepanat)
        src_surf_dir = Path('')
        surf_info_dir = Path('')
        headmodel_dir = anat_dir /f'sub-{hmsub}/HeadModel'
        searchgrid_dir = anat_dir /f'sub-{hmsub}/SearchGrid'
        out_dir = anat_dir /f'sub-{hmsub}/Simulation'
        cmd = [
        'contarg',
        'normgrid',
        'sim-gyral-lip',
        '--headmodel-dir',
        headmodel_dir.as_posix(),
        '--searchgrid-dir',
        searchgrid_dir.as_posix(),
        '--out-dir',
        out_dir.as_posix(),
        '--src-surf-dir',
        src_surf_dir.as_posix(),
        '--fmriprep-dir',
        Path(fmriprep_root).as_posix(),
        '--bids-dir',
        Path(fmriprep_root).as_posix(),
        '--anat-dir',
        fmriprepanat,
        '--surf_src',
        'fmriprep',
        '--njobs',
        "20"
        ]
        cmds.append(' '.join(cmd))
    """
for subject in subjects:
    headmodel_dir = anat_dir /f'sub-{subject}/HeadModel'
    searchgrid_dir = anat_dir /f'sub-{subject}/SearchGrid'
    out_dir = anat_dir /f'sub-{subject}/Simulation'
    src_surf_dir = liston_root / f'sub-{subject}/anat/T1w/fsaverage_LR32k/'

    cmd = [
        'contarg',
        'normgrid',
        'sim-gyral-lip',
        '--headmodel-dir',
        headmodel_dir.as_posix(),
        '--searchgrid-dir',
        searchgrid_dir.as_posix(),
        '--out-dir',
        out_dir.as_posix(),
        '--src-surf-dir',
        src_surf_dir.as_posix(),
        '--njobs',
        "20"
    ]
    cmds.append(' '.join(cmd))
 
      --headmodel-dir PATH     Path to HeadModel directory.  [required]
  --searchgrid-dir PATH    Path to SearchGrid directory.  [required]
  --out-dir PATH           Path to write to.  [required]
  --src-surf-dir PATH      Path in which to find subject surfaces.  [required]
  --coil TEXT              Name of the coil if it's one of the ones that comes
                           with simnibs, othewise the path to it.  [default:
                           MagVenture_MCF-B65.ccd]
  --distancetoscalp FLOAT  Distance in mm from the coil to the scalp.
                           Default=2.  [default: 2]
  --njobs INTEGER          Number of jobs to run in parallel to find targets
                           [default: 1]
  --surf_src TEXT          flag to indicate where data is coming from, options
                           are liston or fmriprep  [default: liston]
  --bids-dir PATH          Bids directory
  --fmriprep-dir PATH      FMRIPREP directory
  --anat-dir PATH          contarg anat outputs directory
"""

NDARINV1285PMCK
NDARINV1EECRFPM
NDARINV1H7JEJW1
NDARINV52XG9LJ3
NDARINV6UU2L6YJ
NDARINV7BTZH7L2
NDARINV7TV9UT40
NDARINV85UUUHN0
NDARINVC25XBH2T
NDARINVCVZ70012
NDARINVD5FWJDCY
NDARINVFPFM6B57
NDARINVG8VWC1TN
NDARINVGH5DHURH
NDARINVLMU4CC2J
NDARINVN394NEWK
NDARINVP9Y862GP
NDARINVPE5JBKJ3
NDARINVRZL7PGK1
NDARINVV104HFJY
NDARINVWFN7K4C5
NDARINVWP2U8M5Y
NDARINVX7KKFJ5E
NDARINVXZRDA6CJ
NDARINVYKVPYY2M


In [54]:
print(len(cmds))
print(anat_dir)

25
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc


In [55]:
#liston_dir = Path('/data/EDB/TMSpilot/liston/')
outdir = Path('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2')
swarm_cmd_dir = outdir /'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = outdir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [56]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'gyral_opts'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'gyral_opts'
    jobid = ! swarm -f {swarm_cmd_file} -g 200 -t 22 --gres=lscratch:100 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

['24032565']


# Run simulations for positional uncertainty
requires gyral lip simulations to be finished before running

## set up dataframes with parameters for uncertainty runs

In [57]:

coil='MagVenture_MCF-B65.ccd'

if not Path(coil).exists():
    simnibs_coil_dir = Path(resource_filename('simnibs', 'resources/coil_models'))
    coil_path = simnibs_coil_dir / f'Drakaki_BrainStim_2022/{coil}'
    if not coil_path.exists():
        coil_path = simnibs_coil_dir / f'legacy_and_other/{coil}'
        if not coil_path.exists():
            raise FileNotFoundError(f"Could not find coil ({coil}) in {simnibs_coil_dir}.")
else:
    coil_path = Path(coil)

In [58]:
nsims = 1000
uncert_deviations_path = anat_dir / f'uncert_deviations{nsims}.npy'
if not uncert_deviations_path.exists():
    uncert_deviations = make_uncert_sims(5, nsims, dist_std, angle_std)
    np.save(uncert_deviations_path, uncert_deviations)
vine = get_prob_vine(4, dist_std, angle_std)

In [66]:
jobs = []
all_subjects = []
for subject in subjects:
    if not Path(anat_dir / f'sub-{subject}').exists():
            continue
    if subject in ["NDARINV1285PMCK"]:#subject failed previos step
        continue
    sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-00')
    if not sim_dir.exists():
        sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-01')
        if not sim_dir.exists():
            print(f"{sim_dir} not found, skipping")
            continue
    headmodel_dir = anat_dir / f'sub-{subject}/HeadModel'
    jobs.append(delayed(setup_uncert_sims)(headmodel_dir, sim_dir, dist_std=dist_std, angle_std=angle_std, outname=f'uncert{nsims}', uncert_deviations_path=uncert_deviations_path))
    all_subjects.append(subject)
    

/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1EECRFPM/Simulation/simulation-01 not found, skipping
/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1H7JEJW1/Simulation/simulation-01 not found, skipping


In [67]:
all_settings = Parallel(n_jobs=8, verbose=10)(jobs)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done  10 out of  22 | elapsed: 13.8min remaining: 16.5min
[Parallel(n_jobs=8)]: Done  13 out of  22 | elapsed: 17.2min remaining: 11.9min
/vf/users/ENIGMA-MDD/code/contargenv/lib/python3.9/site-packages/mixedvines/copula.py:572: RuntimeWarning: invalid value encountered in multiply
  vals[neqz] = norm.cdf((nrvs[:, 0] - self.theta * nrvs[:, 1])
[Parallel(n_jobs=8)]: Done  16 out of  22 | elapsed: 20.4min remaining:  7.7min
[Parallel(n_jobs=8)]: Done  19 out of  22 | elapsed: 22.0min remaining:  3.5min


Calculating settings for 147 simulations.
0, 25, 50, 75, 100, 125, Calculating settings for 114 simulations.
0, 25, 50, 75, 100, Calculating settings for 155 simulations.
0, 25, 50, 75, 100, 125, 150, Calculating settings for 112 simulations.
0, 25, 50, 75, 100, Calculating settings for 77 simulations.
0, 25, 50, 75, Calculating settings for 91 simulations.
0, 25, 50, 75, Calculating settings for 115 simulations.
0, 25, 50, 75, 100, 

[Parallel(n_jobs=8)]: Done  22 out of  22 | elapsed: 26.2min finished


Calculating settings for 70 simulations.
0, 25, 50, Calculating settings for 118 simulations.
0, 25, 50, 75, 100, Calculating settings for 109 simulations.
0, 25, 50, 75, 100, Calculating settings for 112 simulations.
0, 25, 50, 75, 100, Calculating settings for 93 simulations.
0, 25, 50, 75, Calculating settings for 95 simulations.
0, 25, 50, 75, Calculating settings for 97 simulations.
0, 25, 50, 75, Calculating settings for 105 simulations.
0, 25, 50, 75, 100, Calculating settings for 134 simulations.
0, 25, 50, 75, 100, 125, Calculating settings for 126 simulations.
0, 25, 50, 75, 100, 125, Calculating settings for 99 simulations.
0, 25, 50, 75, Calculating settings for 141 simulations.
0, 25, 50, 75, 100, 125, Calculating settings for 134 simulations.
0, 25, 50, 75, 100, 125, Calculating settings for 116 simulations.
0, 25, 50, 75, 100, Calculating settings for 137 simulations.
0, 25, 50, 75, 100, 125, 

In [68]:
print(len(all_settings))
print(len(all_subjects))

22
22


In [69]:
cmds = []
for (subject, settings_path) in zip(all_subjects, all_settings):
    settings = pd.read_pickle(settings_path)
    #print(f'{subject} has {settings}')
    if not Path(anat_dir / f'sub-{subject}').exists():
            continue
    if subject in ["NDARINV1285PMCK"]:#subject failed previos step
        continue
    for oix in sorted(settings.oix.unique()):
        sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-00')
        if not sim_dir.exists():
            sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-01')
            if not sim_dir.exists():
                print(f"{sim_dir} not found, skipping")
        
        HeadModel_dir = anat_dir / f'sub-{subject}/HeadModel'
        try:
            m2m_dir = sorted(HeadModel_dir.glob('m2m*'))[0]
        except IndexError:
            raise FileNotFoundError(f"No m2m directory found in {HeadModel_dir}")
        headmesh_path = m2m_dir / f'{subject}.msh'
        
        uncert_dir = settings_path.parent
        uncert_out = uncert_dir / f'oix-{oix:04d}_stat-abovethreshactprobs_magnE.nii.gz'
        if not uncert_out.exists() or overwrite:
            cmd = [
                'contarg',
                'normgrid',
                'sim-uncert',
                '--headmesh-path',
                headmesh_path.as_posix(),
                '--settings-path',
                settings_path.as_posix(),
                '--ix',
                f'{oix}',
                '--coil-path',
                coil_path.as_posix(),
                '--tmp-dir',
                f'/lscratch/$SLURM_JOB_ID/uncertsim-{oix}',
                '--out-dir',
                settings_path.parent.as_posix(),
                '--njobs',
                f'20',
                '--thresh-type=mt',
                f'--min-thresh={mt_thresh}',
                f'--max-thresh={maxMT}',
                f'--max-mt={maxMT}',
            ]
            cmds.append(' '.join(cmd))

In [70]:
cmds[22]

'contarg normgrid sim-uncert --headmesh-path /data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV52XG9LJ3/HeadModel/m2m_NDARINV52XG9LJ3/NDARINV52XG9LJ3.msh --settings-path /data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV52XG9LJ3/Simulation/simulation-00/uncert1000/settings.pkl.gz --ix 22 --coil-path /vf/users/ENIGMA-MDD/code/contargenv/lib/python3.9/site-packages/simnibs/resources/coil_models/Drakaki_BrainStim_2022/MagVenture_MCF-B65.ccd --tmp-dir /lscratch/$SLURM_JOB_ID/uncertsim-22 --out-dir /data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV52XG9LJ3/Simulation/simulation-00/uncert1000 --njobs 20 --thresh-type=mt --min-thresh=60 --max-thresh=80 --max-mt=80'

In [71]:
len(cmds)

2497

In [75]:
# deal with 1000 job swarm limit
# Use while loop instead.

if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'uncert_sims'
    print(swarm_cmd_file)
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'uncert_sims'
    #jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 8:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    #print(jobid)

/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/swarm/swarm_cmds/uncert_sims


In [22]:
if len(cmds) > 1000:
    swarm_cmd_file = swarm_cmd_dir / 'uncert_sims'
    swarm_cmd_file.write_text('\n'.join(cmds[1000:]))
    run_name = 'uncert_sims'
    jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 8:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

['19610544']


## Here's code for restarting failues
For currently unknown reasons, some jobs run slowly and time out. When rerun, those jobs complete without issue.

In [ ]:
to_restart = [
    cmds[428],
    cmds[209],
    cmds[185],
    cmds[181],
    cmds[137],
    cmds[114],
    cmds[45],
    cmds[41],
    cmds[35],
]
to_restart

In [ ]:
if len(to_restart) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'retart_uncert_sims'
    swarm_cmd_file.write_text('\n'.join(to_restart))
    run_name = 'restart_uncert_sims'
    jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

 # Create surfaces from uncertainty sims

In [76]:
all_settings = []
all_subjects = []
for subject in subjects:
    if not Path(anat_dir / f'sub-{subject}').exists():
            continue
    if subject in ["NDARINV1H7JEJW1","NDARINV1285PMCK","NDARINV7BTZH7L2"]:#subject failed previos step
            continue
    sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-00')
    if not sim_dir.exists():
        sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-01')
        if not sim_dir.exists():
            print(f"{sim_dir} not found, skipping")
    uncert_dir = sim_dir / f'uncert{nsims}'
    settings_path = uncert_dir / 'settings.pkl.gz'
    all_settings.append(settings_path)
    all_subjects.append(subject)

/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1EECRFPM/Simulation/simulation-01 not found, skipping


In [77]:
print(len(all_settings))
print(len(all_subjects))

22
22


In [78]:
for (subject, settings_path) in zip(all_subjects, all_settings):
    print(subject)
    if not Path(anat_dir / f'sub-{subject}').exists():
            continue
    if subject in ["NDARINV1H7JEJW1","NDARINV1285PMCK","NDARINV7BTZH7L2"]:#subject failed previos step
        continue
    outfile = f'/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-{subject}/Simulation/simulation-00/uncert1000/sub-{subject}_space-fsLR_den-32k_desc-abovethreshactprobs_stat.dtseries.nii'
    #print(Path(outfile))
    if Path(outfile).is_file():
        print('this exists')

NDARINV1EECRFPM
NDARINV52XG9LJ3
NDARINV6UU2L6YJ
NDARINV7TV9UT40
NDARINV85UUUHN0
NDARINVC25XBH2T
NDARINVCVZ70012
NDARINVD5FWJDCY
NDARINVFPFM6B57
NDARINVG8VWC1TN
NDARINVGH5DHURH
NDARINVLMU4CC2J
NDARINVN394NEWK
NDARINVP9Y862GP
NDARINVPE5JBKJ3
NDARINVRZL7PGK1
NDARINVV104HFJY
NDARINVWFN7K4C5
NDARINVWP2U8M5Y
NDARINVX7KKFJ5E
NDARINVXZRDA6CJ
NDARINVYKVPYY2M


In [83]:

jobs = []
for (subject, settings_path) in zip(all_subjects, all_settings):
    if not Path(anat_dir / f'sub-{subject}').exists():
            continue
    if subject in ["NDARINV1H7JEJW1","NDARINV1285PMCK","NDARINV7BTZH7L2","NDARINV1EECRFPM","NDARINV52XG9LJ3", "NDARINV6UU2L6YJ", "NDARINV7TV9UT40"]:#subject failed previos step
        continue
    outfile = f'/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-{subject}/Simulation/simulation-00/uncert1000/sub-{subject}_space-fsLR_den-32k_desc-abovethreshactprobs_stat.dtseries.nii'
    if Path(outfile).exists():
        continue
    filesT1w =  bidslayout.get(subject=subject,
                                         #session=session,
                                         extension=".nii.gz",
                                         suffix="T1w",
                                         desc="preproc",
                                         space=None)
    fmriprepanat = filesT1w[0].dirname
    #src_surf_dir = liston_root / f'sub-{subject}/anat/T1w/fsaverage_LR32k/'
    uncert_dir = settings_path.parent
    #jobs.append(delayed(make_uncert_surfaces)(subject, Path(fmriprepanat), uncert_dir, overwrite=True, fmriprep=True,
    #                                          anat_dir=Path(fmriprepanat)))
    #print(subject)
    make_uncert_surfaces(subject, Path(fmriprepanat), uncert_dir, overwrite=True, fmriprep=True,
                                              layout=bidslayout,anat_dir=Path(fmriprepanat))


++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 16.5 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 16.6 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 12.1 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 47.9 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 47.6 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 36.6 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of output dataset to 1.0 s
++ elapsed time = 69.4 s
++ 3dTcat: AFNI version=AFNI_24.1.02 (Apr  8 2024) [64-bit]
*+ WARNING: Set TR of o

In [85]:
print(uncert_dir)

/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/anat_preproc/sub-NDARINV1EECRFPM/Simulation/simulation-01/uncert1000


In [14]:
#Parallel(n_jobs=10, verbose=10)(jobs)#PArallel not working with bidslayout. I'll just for loop.
    

# Run repdist clustering on time series so we can get stats for each position

This can be run in parallel with the gyral lip and uncertainty simulations above 

In [13]:
layout = '/data/ENIGMA-MDD/BIDS/ABCD/derivatives/layout'
#bidslayout.save('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/layout')
#layout2 = bids.BIDSLayout( database_path=layout)
#layout2.files

In [ ]:
jobs = []
clusts = []
outdir = Path('/data/ENIGMA-MDD/BIDS/ABCD/derivatives/contarg_fmriprep_23-2/')
smoothings = ['2.55']
for subj in subjects: 
    if subj in ["NDARINV1H7JEJW1","NDARINV1285PMCK","NDARINV7BTZH7L2","NDARINV1EECRFPM","NDARINV52XG9LJ3", "NDARINV6UU2L6YJ", "NDARINV7TV9UT40"]:#subject failed previos step
        continue
    print(subj)
    
    for session in bidslayout.get_sessions(subject=subj):
        for smoothing in smoothings:
            subj_func_outdir =  outdir / f'func_preproc/sub-{subj}/'
            ses_func_outdir = subj_func_outdir /  f'ses-{session}'
            clust_outdir = ses_func_outdir / 'cluster'
            #lrest_dir = liston_root / f'sub-{subject}/func/rest'
            #concat_nii = lrest_dir / f'session_{session}/concatenated/Rest_session-{session}_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'
            #src_surf_dir = liston_root / f'sub-{subject}/anat/T1w/fsaverage_LR32k'
            filesT1w =  bidslayout.get(subject=subj,
                                         #session=session,
                                         extension=".nii.gz",
                                         suffix="T1w",
                                         desc="preproc",
                                         space=None)
            fmriprepanat = filesT1w[0].dirname
            if len(bidslayout.get_runs(subject=subj, session=session)) == 0:
                    runs = [None]
            else:
                    runs = bidslayout.get_runs(subject=subj, session=session)
            for r in runs: 
                    print(r)
                    func = bidslayout.get(subject=subj,
                                                         session=session,
                                                         extension=".dtseries.nii",
                                                         suffix="bold",
                                                         #hemi=hemi,
                                                         run = r,
                                                         desc='AROMAGW',
                                                         space='fsLR',
                                                         regex_search=False,
                                                         invalid_filters="allow",
                                                         return_type='filename')
                    if len(func) < 1:
                        continue
                    run_clusters(
                    subj,
                    Path(func[0]),
                    clust_outdir,
                    Path(fmriprepanat),
                    surf_source='fmriprep',
                    out_prefix=f'sub-{subj}_ses-{session}_run-{r}_smoothing-{smoothing}_',
                    overwrite=True,
                    layout=bidslayout,
                    anat_dir=Path(fmriprepanat)
                    )
            
                    #jobs.append(delayed(run_clusters)(
                    #            subj,
                    #            Path(func[0]),
                    #            clust_outdir,
                    #            Path(fmriprepanat),
                    #            surf_source='fmriprep',
                    #            out_prefix=f'sub-{subj}_ses-{session}_run-{r}_smoothing-{smoothing}_',
                    #            overwrite=True,
                    #            layout=layout,
                    #            anat_dir=Path(fmriprepanat)
                    #            ))
                    clusts.append(dict(
                                    subject = subj,
                                    session = session,
                                    run=r,
                                    smoothing = smoothing
                                    ))


#subject, concat_nii, clust_outdir, src_surf_dir,
#                 surf_source='liston',
#                 maxrepdist=0.2,
#                 medial_wall=None,
#                 refroi='bilateralfullSGCsphere',
#                 stimroi='expandedcoleBA46',
#                 out_prefix='',
#                 layout=None,
#                 anat_dir=None,
#                 overwrite=False

NDARINV1EZ26N40
12
13
5
6
7
10
3
NDARINV2484CB0H
11


/vf/users/ENIGMA-MDD/code/contargenv/lib/python3.9/site-packages/statsmodels/stats/multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


4
5
6
3
9
45
611
9999


/vf/users/ENIGMA-MDD/code/contarg/contarg/clustering.py:581: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(2, 1, figsize=(5, 10))


NDARINV2BW6G83G
13
14
6
7
10
3
1314
67
9999
NDARINV7A5RDHVW
13
14
7
8
11
3
4
5
1314
78
9999
NDARINV85UUUHN0
4
10
21
23
8
17
2123
810
9999
NDARINVC25XBH2T
3
20
7
9
16
79
9999
NDARINVCVZ70012
4
10
21
8
17
810
9999
NDARINVD5FWJDCY
5
11
22
24
9
18
2224
911
9999
NDARINVEN1P5RFL
11


...

9999
NDARINVRZL7PGK1
4
10
21
8
17
810
9999
NDARINVU3CF21GV
11
5
6
3
9
56
9999
NDARINVU7WWPJ0M
13
14
6


/vf/users/ENIGMA-MDD/code/contargenv/lib/python3.9/site-packages/statsmodels/stats/multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


7
10
3
1314
67
9999
NDARINVV104HFJY
4
10


/vf/users/ENIGMA-MDD/code/contargenv/lib/python3.9/site-packages/statsmodels/stats/multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


21
23
8
17
2123
810
9999
NDARINVWFN7K4C5
4
12
14
25
27
8
21
1425
812
9999
NDARINVWP2U8M5Y
4
10
21
23
8
17
2123
810
9999
NDARINVX7KKFJ5E
4
10
21
26
8
17
2126
810
9999
NDARINVXTMW4PWB
3
5
NDARINVXZRDA6CJ
4
10


In [15]:
clustreses = Parallel(n_jobs=10, verbose=10)(jobs)
# Parallel won't work with pybids.
# One solution is to save and load in the functions... need more time to implement.

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


TypeError: root argument must be a pathlib.Path (or a type that supports casting to pathlib.Path, such as string) specifying the directory containing the BIDS dataset.